In [3]:
import pandas as pd

In [4]:
#Чтение файлов:
df1T = pd.read_excel('Источники/transactions_data.xlsx')
df2C = pd.read_json('Источники/clients_data.json')  

In [5]:
#Проверки на дубли:
df1T.duplicated().sum()
df2C.duplicated().sum()

np.int64(0)

In [7]:
#Проверка на пустые значения:
df1T.isna().sum() #Проверка на Пустые значения в Транзакциях. 

#UPD: Значения по 0 должно быть, после очистки. UPD: Есть данные где остальные присутствуют, но нет дат. 

transaction_id        0
client_id             0
transaction_date    789
service               0
amount                0
payment_method        0
city                  0
consultant            0
dtype: int64

In [8]:
#Проверка на пустые значения:
df2C.isna().sum() #Проверка на Пустые значения в Клиентах.

id              1
age           983
gender       3319
net_worth     475
dtype: int64

In [9]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Провоерка выше.

In [10]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

Пробуем сначала преобразовать даты, есть в строках со значением "INVALID_DATE". 
Возможно действие не эффективно и не целесообразное, но мы проверим. 

In [12]:
#Преобразование дат к одному в формату с целью проверки изменения значения:
df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'])
#Приведение всех дат к одному формату с точностью до секунд:
df1T['transaction_date'] = df1T['transaction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

#*Чтобы сработало нужно ниже выполнить сначала "#Замена INVALID_DATE на Nan".

In [11]:
#Замена INVALID_DATE на Nan

df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'], errors='coerce')

In [13]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

Есть файлы где нет даты, но остальные в норме. Продолжим удаление, но по условиям. 

In [14]:
#Удаление строк с "Неизвестной услугой":
df1T = df1T[df1T['service'] != 'Неизвестная услуга']

In [15]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [16]:
#Продолжаем очистку, теперь зайдем со стороны аномальных значений:
#Использую "Межквартильный размах" (IQR). 

# Рассчитываем первый и третий квартели, чтобы посчитать IQR.
Q1 = df1T['amount'].quantile(0.25)
Q3 = df1T['amount'].quantile(0.75)

#Расчитываем IQR:
IQR = Q3 - Q1

#Устанавливаем границы 
lower_bound = max(0, Q1 - 1.5 * IQR) #Нижняя. Нижняя не может быть меньше 0. (...отрицательные суммы).
upper_bound = Q3 + 1.5 * IQR #Верхняя.

#Вывод границ: 
print(lower_bound)
print(upper_bound)

0
145125.1343807844


In [18]:
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]
print(anomal[['service','amount']])

                              service         amount
25            Финансовое планирование  147439.369086
461   Инвестиционное консультирование  145713.605919
505            Налоговое планирование  149407.108459
638   Инвестиционное консультирование  146218.600215
1000           Налоговое планирование  147333.389591
1685  Инвестиционное консультирование  148549.499778
2708        Структурирование капитала  145156.846847
3088              Управление активами  145912.853299
3142          Финансовое планирование  147667.516120
3263          Финансовое планирование  148470.649643
3425          Финансовое планирование  147064.285766
3945        Структурирование капитала  145827.584354
4581  Инвестиционное консультирование  146821.816904
5595           Налоговое планирование  146609.123497
6085  Инвестиционное консультирование  148873.575903
6212  Инвестиционное консультирование  147194.185379
6471  Инвестиционное консультирование  147704.029797
7049  Инвестиционное консультирование  145955.

In [21]:
#Удаление аномальных значений: 
df1T = df1T[(df1T['amount'] >= lower_bound) & (df1T['amount'] <= upper_bound)] 

In [22]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [23]:
#Проверка: 
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]

print(anomal)

Empty DataFrame
Columns: [transaction_id, client_id, transaction_date, service, amount, payment_method, city, consultant]
Index: []
